# Module 1, Section 3: Multi-Agent Architecture


In this section, you'll learn to build a multi-agent customer support system using:
- **Specialized sub-agents** focused on distinct domains (database vs. documents)
- **Supervisor agent** that smartly routes queries to the right expert
- **Tool wrapping**, enabling the supervisor to delegate tasks to sub-agents as tools
- **Testing with LangSmith traces** to see multi-agent coordination in action

By the end, you'll have a working system with:
- **Database Agent** for order, product, and customer queries
- **Documents Agent** for searching product documents and policies
- **Supervisor** for orchestration and delegation

<div align="center">
    <img src="../../static/supervisor_agent.png">
</div>


## Setup

Load environment variables:

In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Import Tools

We'll use the tools we created in previous sections, plus new Documents tools for document search:

<div align="center">
    <img src="../../static/db_rag_tools.png" alt="Schema Diagram">
</div>

In [ ]:
# Import database tools (from Section 1)
from tools import get_order_status, get_product_price, get_order_items

# Import Documents tools (new in Section 3)
from tools import search_product_docs, search_policy_docs

### What is RAG (Retrieval-Augmented Generation)?

Both our Database and Documents agents use retrieval patterns - they retrieve information to augment the LLM's context. The Documents Agent specifically uses **RAG with semantic search** over unstructured documents:

1. **Documents are split into chunks** (~1000 characters)
2. **Chunks become vectors (embeddings)** that capture meaning
3. **User query becomes a vector** too
4. **Similar vectors = similar meaning** → retrieve relevant chunks
5. **LLM uses retrieved chunks** to answer the question

**Our Setup:**
- **25 product docs + 5 policy docs** → **337 chunks**
- **HuggingFace embeddings** (local, no API key needed)
- **Metadata filtering** separates products from policies
- **Pre-built vectorstore** loaded from `data/vector_stores/`

The Documents tools (`search_product_docs` and `search_policy_docs`) wrap this complexity into simple function calls!

In [3]:
result = search_policy_docs.invoke("Whats your return policy?")
print(result)

[return_policy]
# Return Policy

At TechHub, we want you to be completely satisfied with your purchase. If you're not happy with your order, we accept returns within our specified return windows.

## Return Windows

**Unopened Electronics**
- 30 days from delivery date
- All original packaging and seals must be intact
- Full refund to original payment method

**Opened Electronics**
- 14 days from delivery date
- Product must be in good working condition
- All accessories, cables, and documentation must be included
- Original packaging helpful but not required

**All Other Items**
- 30 days from delivery date
- Must be in original condition

## Condition Requirements

To qualify for a return, items must meet the following conditions:

---

[return_policy]
**All Other Items**
- 30 days from delivery date
- Must be in original condition

## Condition Requirements

To qualify for a return, items must meet the following conditions:

- Product is in good working condition with no physical da

## 2. Build Database Agent

Our first specialist: an agent focused on querying structured data (orders, products, customers).

In [4]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver

# Initialize model
llm = init_chat_model("anthropic:claude-haiku-4-5")

# Create Database Agent
db_agent = create_agent(
    model=llm,
    tools=[get_order_status, get_product_price, get_order_items],
    system_prompt="""You are a database specialist for TechHub customer support.
    
Your role is to query:
- Order status and details
- Product prices and availability
- Items purchased in specific orders

Always provide specific, accurate information from the database.
If you cannot find information, say so clearly.""",
    checkpointer=MemorySaver(),
)

Let's test the Database Agent:

In [5]:
import uuid

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}


result = db_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What items were in order ORD-2024-0063?"}
        ]
    },
    config=config,
)

print(result["messages"][-1].content)

Order ORD-2024-0063 contained the following item:

- **Dell UltraSharp 27" 4K Monitor** (Product ID: TECH-MON-006)
  - Quantity: 1
  - Price: $543.74

Is there anything else you'd like to know about this order?


## 3. Build Documents Agent

Our second specialist: an agent focused on searching product documentation and policies.

In [ ]:
# Create Documents Agent
docs_agent = create_agent(
    model=llm,
    tools=[search_product_docs, search_policy_docs],
    system_prompt="""You are a company policy and product information specialist for TechHub customer support.

Your role is to answer questions about product specifications, features, compatibility,
policies (returns, warranties, shipping), and setup instructions.

Always search the documentation to provide accurate, detailed information.""",
    checkpointer=MemorySaver(),
)

Let's test the Documents Agent:

In [ ]:
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = docs_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's your return policy for opened electronics?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's your return policy for opened electronics?
================================== Ai Message ==================================

[{'id': 'toolu_01CdwcCbnPKnBz5VU4NQX7BW', 'input': {'query': 'return policy opened electronics'}, 'name': 'search_policy_docs', 'type': 'tool_use'}]
Tool Calls:
  search_policy_docs (toolu_01CdwcCbnPKnBz5VU4NQX7BW)
 Call ID: toolu_01CdwcCbnPKnBz5VU4NQX7BW
  Args:
    query: return policy opened electronics
================================= Tool Message =================================
Name: search_policy_docs

[return_policy]
# Return Policy

At TechHub, we want you to be completely satisfied with your purchase. If you're not happy with your order, we accept returns within our specified return windows.

## Return Windows

**Unopened Electronics**
- 30 days from delivery date
- All original packaging and seals must be intact
- Full refund to original payment method

**Opened 

## 4. Build Supervisor Agent

Now we'll create a supervisor agent that coordinates our specialists.

**Key insight:** Sub-agents become *tools* for the supervisor!

In [ ]:
from langchain_core.tools import tool


# Wrap Database Agent as a tool
@tool
def database_specialist(query: str) -> str:
    """Consult the database specialist for structured data queries.

    This specialist has access to TechHub's operational database and can:
    - Look up order status and tracking information
    - Retrieve customer order history
    - Check product prices and inventory availability

    Use this specialist when the user needs real-time operational data
    (orders, inventory, pricing) rather than product details or policies.

    Args:
        query: The question to ask the database specialist (may be rephrased by supervisor)

    Returns:
        The database specialist's answer based on current database records
    """
    result = db_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content


# Wrap Documents Agent as a tool
@tool
def documentation_specialist(query: str) -> str:
    """Consult the documentation specialist for product information and policies.

    This specialist has access to TechHub's documentation library and can:
    - Look up product specifications, features, and technical details
    - Explain return, refund, and warranty policies
    - Provide shipping and delivery information
    - Answer questions about product compatibility and setup

    Use this specialist when the user needs detailed product information,
    policy explanations, or guidance (not real-time operational data).

    Args:
        query: The question to ask the documentation specialist (may be rephrased by supervisor)

    Returns:
        The documentation specialist's answer based on product docs and policies
    """
    result = docs_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content

**💡 Best Practice - Naming & Describing Subagent Tools:**

The tool **names** and **descriptions** are critical for supervisor routing:

1. **Names** (`database_specialist`, `documentation_specialist`):
   - Clear and descriptive (not generic like `query_db`)
   - Indicate these are specialized agents, not simple tools
   - Influence how the LLM reasons about when to use them

2. **Descriptions**:
   - Explicitly list capabilities ("can look up X, Y, Z")
   - Provide clear routing guidance ("Use this when...")
   - Help the supervisor make better decisions

The supervisor only sees these tool signatures - it never sees the sub-agent's internal prompts or tools!

In [9]:
# Create Supervisor Agent
supervisor_agent = create_agent(
    model=llm,
    tools=[database_specialist, documentation_specialist],
    system_prompt="""You are a supervisor for TechHub customer support.

Your role is to understand customer questions and route them to the appropriate specialists with additional context as needed:
- Use database_specialist for order status, product prices, and customer order history
- Use documentation_specialist for product specs, policies, and general information

You can use multiple tools if needed to fully answer the question.
Always provide helpful, complete responses to customers.""",
    checkpointer=MemorySaver(),
)

## 5. Test Simple Routing

Let's test the supervisor with queries that need just ONE specialist:

In [10]:
print("Query 1: Order status (should route to Database Agent)")
print("=" * 60)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2025-0030?"}
        ]
    },
    config=config,
)

result["messages"][-1].pretty_print()
print("\n💡 Check LangSmith traces to see: Supervisor → database_specialist → DB Agent")

Query 1: Order status (should route to Database Agent)
================================== Ai Message ==================================

Your order **ORD-2025-0030** is currently in **Processing** status. It has not yet been shipped, so there's no tracking number available yet. 

You'll receive an email with tracking information as soon as your order ships. If you have any other questions about your order, feel free to ask!

💡 Check LangSmith traces to see: Supervisor → database_specialist → DB Agent


In [ ]:
print("\nQuery 2: Product question (should route to Documents Agent)")
print("=" * 60)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's included in the box with the Logitech MX Keys keyboard?",
            }
        ]
    },
    config=config,
)

result["messages"][-1].pretty_print()
print(
    "\n💡 Check LangSmith traces to see: Supervisor → documentation_specialist → Documents Agent"
)


Query 2: Product question (should route to RAG Agent)
================================== Ai Message ==================================

The **Logitech MX Keys keyboard** box includes:

1. **The Logitech MX Keys Wireless Keyboard** itself
2. **Logitech Bolt USB receiver** - for wireless connectivity
3. **USB-A to USB-C charging cable** (1.5m) - for charging the keyboard
4. **User documentation** - setup and usage guides

The keyboard comes pre-charged, so you can start using it right away, though an initial full charge (around 3 hours) is recommended before first use. You can also use the keyboard while it's charging.

Additionally, you can download the **Logitech Options+ software** for free from logitech.com if you want advanced customization options, though it's not included in the box.

💡 Check LangSmith traces to see: Supervisor → documentation_specialist → RAG Agent


## 6. Test Multi-Agent Coordination

Now the more interesting part - queries that require BOTH sub-agents!

**Example:** Query that requires both Database AND Documents subagents, can run with PARALLEL execution

In [ ]:
print("Query 3: Requires both Database AND Documents agents - parallel")
print("=" * 60)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Is the MacBook Air in stock? What type of processor does it have? And if I buy it, what's the return policy?",
            }
        ]
    },
    config=config,
)

result["messages"][-1].pretty_print()
print("\n💡 Check LangSmith traces to see:")
print(
    "   Supervisor → database_specialist → DB Agent → documentation_specialist → Documents Agent"
)

Query 3: Requires both Database AND RAG agents - parallel
================================== Ai Message ==================================

Perfect! Here's the complete information about the MacBook Air:

## Stock Status
✅ **Yes, the MacBook Air M2 (13-inch, 256GB) is in stock** at TechHub for **$1,199.00**

## Processor Details
The MacBook Air features the **Apple M2 chip** with:
- **8-core CPU** for powerful processing
- **8-core GPU** for excellent graphics performance
- **8GB unified memory** (shared between CPU and GPU)
- The M2 offers excellent performance while maintaining efficiency and includes a fanless design for silent operation
- Battery life of up to 18 hours

## Return Policy
- **If Unopened:** 30 days from delivery with full refund, no restocking fee
- **If Opened:** 14 days from delivery (must be in good condition with all accessories included)
- **Condition Requirements:** Product must work properly, include all original accessories, and show no signs of misuse or phy

**Example:** Query that requires SEQUENTIAL agent execution - output from first agent feeds into second agent

This demonstrates true agent orchestration where the supervisor can't parallelize!

In [ ]:
print("Query 4: Requires SEQUENTIAL coordination (DB → Documents)")
print("=" * 60)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I bought a monitor in my last order (ORD-2024-0063). Is the MacBook Air compatible with it?",
            }
        ]
    },
    config=config,
)

result["messages"][-1].pretty_print()
print("\n💡 Check LangSmith traces to see SEQUENTIAL flow:")

Query 4: Requires SEQUENTIAL coordination (DB → RAG)
================================== Ai Message ==================================

Perfect! **Yes, your Dell UltraSharp 27" 4K Monitor is fully compatible with MacBook Air.** 

Here's what you need to know:

**Best Connection Option:** Use the included **USB-C cable** - it will connect your monitor to your MacBook Air and also charge it at 65W simultaneously, so you won't need a separate power adapter.

**Alternative Options:** The monitor also comes with HDMI and DisplayPort cables if you prefer those connections.

The monitor comes with all the necessary cables included in the box, so you're all set to connect it right away. The setup is straightforward - just plug in the USB-C cable between your MacBook Air and the monitor, and you're ready to go!

Is there anything else you'd like to know about setting up your monitor?

💡 Check LangSmith traces to see SEQUENTIAL flow:


### Try More Examples

Test the supervisor with various queries to see routing in action:

In [ ]:
# Try these queries and watch the traces in LangSmith!

test_queries = [
    # ===== SIMPLE ROUTING (Single Agent) =====
    # Database Agent only
    "What's the status of order ORD-2024-0123?",
    "How much does the MacBook Air cost and is it in stock?",
    "What items did I order in ORD-2024-0063?",
    # Documents Agent only (product specs)
    "What are the specs of the Sony WH-1000XM5 headphones?",
    "Tell me about the ports on the Dell UltraSharp monitor",
    # Documents Agent only (policies)
    "What's your return policy?",
    "How long is the warranty on electronics?",
    # ===== PARALLEL COORDINATION (Both Agents Independently) =====
    "Is the MacBook Air in stock? What processor does it have?",
    "How much is the Logitech MX Keys and what are its key features?",
    # ===== SEQUENTIAL COORDINATION (Output → Input) =====
    # DB → DB: Get order items, then check availability
    "What did I buy in order ORD-2024-0063 and is it still available?",
    # DB → Documents: Get order items, then get specs
    "I ordered something in ORD-2023-0023. Can you tell me about its features?",
    # DB → Documents: Get order items, then get compatibility/policy info
    "I bought a monitor in order ORD-2024-0063. Is the MacBook Air compatible with it?",
]

# Uncomment to test:
# for i, query in enumerate(test_queries, 1):
#     print(f"\n[{i}/{len(test_queries)}] Query: {query}")
#     print("=" * 70)
#     result = supervisor_agent.invoke(
#         {"messages": [{"role": "user", "content": query}]},
#         config={"configurable": {"thread_id": str(uuid.uuid4())}}
#     )
#     print(result["messages"][-1].content)
#     print()

print("✓ Try uncommenting the code above to test all queries!")
print(f"\n📊 Total test queries: {len(test_queries)}")
print("\n💡 Query Types:")
print("   • Simple routing: 7 queries (single agent handles entire request)")
print("   • Parallel coordination: 2 queries (both agents work independently)")
print("   • Sequential coordination: 3 queries (one agent's output feeds another)")
print("\n🔍 Watch LangSmith traces to see:")
print("   - Which specialist the supervisor chooses")
print("   - Which tools each specialist uses")
print("   - Sequential dependencies in action!")

---

## 📦 Code Refactoring Note

The agents we built in this section (Database Agent, Documents Agent, and Supervisor) have been **refactored into the `agents/` directory** as reusable factory functions:

- `agents/db_agent.py` - Database Agent factory
- `agents/docs_agent.py` - Documents Agent factory
- `agents/supervisor_agent.py` - Supervisor Agent factory

**Why factory functions?**
- Fresh checkpointer for each instantiation (no state pollution)
- Clean imports and reusability across notebooks
- Standard LangChain pattern

In **Section 4**, we'll import these agents rather than redefining them:
```python
from agents import create_db_agent, create_docs_agent, create_supervisor_agent
```

This demonstrates how to architect production-ready, reusable code! 🏗️

## Key Takeaways

#### What We Built

1. **Specialized Sub-Agents**
   - Database Agent: Expert at structured data queries
   - Documents Agent: Expert at document search
   - Each agent has focused tools and expertise

2. **Supervisor Pattern**
   - Sub-agents wrapped as tools (`@tool` decorator)
   - Supervisor routes queries to appropriate specialist(s)
   - Can orchestrate **parallel** or **sequential** coordination

3. **Coordination Patterns**
   - **Simple routing**: Single agent handles entire query
   - **Parallel coordination**: Multiple agents work independently on different parts
   - **Sequential coordination**: Output from one agent feeds into another (true orchestration!)

4. **Benefits of Multi-Agent Architecture**
   - **Separation of concerns** - Each agent has clear responsibility
   - **Easier debugging** - Traces show which agent handled what
   - **Maintainability** - Update one agent without affecting others
   - **Scalability** - Easy to add new specialist agents


#### Viewing Traces in LangSmith

Go to your LangSmith project to see:
- **Routing decisions** - Which agent(s) the supervisor called
- **Tool executions** - What tools each agent used
- **Message flow** - Complete conversation tree
- **Timing** - How long each step took
- **Sequential dependencies** - See when agents must run in order

This visibility makes multi-agent systems much easier to debug!

#### What's Next: Section 4 (Advanced Track)

In **Section 4**, we'll use **LangGraph primitives** to build even more sophisticated workflows:
- Custom state management beyond messages
- Conditional routing based on state
- `interrupt()` for Human-in-the-Loop
- Customer verification workflows

**Key decision point:**
- Use `create_agent` for standard agent workflows ✅
- Use LangGraph primitives for complex orchestration needs ✅

#### Looking Ahead to Module 2

Notice how we're still limited by having only specific, pre-defined tools:
- Can't answer: "What products has customer CUST-001 bought?" (need SQL JOINs)
- Can't answer: "Show me all orders over $500" (need flexible queries)
- Can't answer: "Which customers bought MacBooks?" (need complex SQL)

In **Module 2**, we'll upgrade the Database Agent to a **SQL Generation Agent** that can:
- Generate dynamic SQL queries for ANY question
- Handle complex JOINs and aggregations
- Dramatically expand system capabilities

This sets up a clear pedagogical win: Module 1 teaches multi-agent patterns, Module 2 shows how to make agents more powerful! 🚀